<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/Final_project/Fernando_Fortes_Granado/Pruning_GPT_J_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Thu Jun 30 17:25:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
torch.cuda.empty_cache()

In [6]:
from transformers import AutoTokenizer, GPTNeoXForCausalLM
from transformers import GPTJForCausalLM, GPTNeoXTokenizerFast
from transformers import GPT2Tokenizer, GPT2Model

import torch

model = GPTJForCausalLM.from_pretrained(
    "EleutherAI/gpt-j-6B", revision="float16", torch_dtype=torch.float16, low_cpu_mem_usage=True
)

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

device = torch.device("cuda")
model.to(device)

KeyboardInterrupt: ignored

In [ ]:
from datasets import load_dataset
import numpy as np

imdb = load_dataset("imdb")
MAX_TOKENS = 300

train_dataset = imdb["train"].shuffle(seed=42)
test_dataset = imdb["test"].shuffle(seed=42)

train_num_tokens = np.array([tokenizer(train_dataset[idx]["text"], return_tensors="pt").input_ids.shape[1] for idx in range(len(train_dataset))])
test_num_tokens = np.array([tokenizer(test_dataset[idx]["text"], return_tensors="pt").input_ids.shape[1] for idx in range(len(test_dataset))])

train_dataset_idx = np.argwhere(train_num_tokens <= MAX_TOKENS).reshape(-1)
test_dataset_idx = np.argwhere(test_num_tokens <= MAX_TOKENS).reshape(-1)

In [ ]:
from typing import *

def get_inference_sample(dataset, example_number, dataset_idx, idx):
  return [f"""Example {example_number}:
Review: {dataset[int(dataset_idx[idx])]["text"]}
Sentiment"""
], "Positive" if dataset[int(dataset_idx[idx])]["label"] else "Negative"

def build_fixed_input(dataset, dataset_idx, learning_sample_idxs):
  return ["Instruction: Given a movie review, answer if the sentiment of the review is positive or negative."] + [f"""Example {i}:
Review: {dataset[int(dataset_idx[idx])]["text"]}
Sentiment: {"Positive" if dataset[int(dataset_idx[idx])]["label"] else "Negative"}""" for i, idx in enumerate(learning_sample_idxs)
]

def predict(prompt, model, tokenizer):
  input_ids = tokenizer(prompt, return_tensors="pt").input_ids
  input_ids = input_ids.to(device)

  with torch.no_grad():
    gen_tokens = model.generate(
        input_ids,
        do_sample=False,
        temperature=1,
        max_length=2048,
    )
  gen_text = tokenizer.batch_decode(gen_tokens)[0]
  return gen_text.split("\n")[len(prompt.split("\n")) - 1].split()[1]

def evaluate(model, tokenizer, learning_samples: str, inference_samples: List[Tuple[str, str]]):
  num_correct_preds = 0
  for idx in range(len(inference_samples)):
    if ((idx % 10) == 0) and (idx != 0):
      print(f"Number of samples evaluated: {idx}, Number of correct preds: {num_correct_preds}, Accuracy: {num_correct_preds / idx}")
    prompt = "\n\n".join(learning_samples + inference_samples[idx][0])
    label = inference_samples[idx][1]
    output = predict(prompt, model, tokenizer)
    print(label, output, label == output)

    if label == output:
      num_correct_preds += 1
    else:
      print("\n_____________________________\n")
      print("PROMPT")
      print(prompt)
      print("\n_____________________________\n")
      print("LABEL")
      print(label)
      print("OUTPUT")
      print(output)
      print("\n_____________________________\n")
  return num_correct_preds / len(inference_samples)


NUM_INFERENCE_SAMPLES = 200
NUM_LEARNING_SAMPLES = 5


learning_samples = build_fixed_input(
    dataset=train_dataset, 
    dataset_idx=train_dataset_idx, 
    learning_sample_idxs=[i for i in range(NUM_LEARNING_SAMPLES)]
)

train_inference_samples = [get_inference_sample(
    dataset=train_dataset,
    example_number=NUM_LEARNING_SAMPLES,
    dataset_idx=train_dataset_idx,
    idx=i
) for i in range(NUM_INFERENCE_SAMPLES + NUM_LEARNING_SAMPLES)]

test_inference_samples = [get_inference_sample(
    dataset=test_dataset,
    example_number=NUM_LEARNING_SAMPLES,
    dataset_idx=test_dataset_idx,
    idx=i
) for i in range(NUM_INFERENCE_SAMPLES)]

In [ ]:
evaluate(model, tokenizer, learning_samples, train_inference_samples)
# Acuracia no treino: 93%, NUM_INFERENCE_SAMPLES: 200

In [ ]:
evaluate(model, tokenizer, learning_samples, test_inference_samples)
# Acuracia no teste: XX%, NUM_INFERENCE_SAMPLES: 200